In [2]:
import os
import pandas as pd

In [4]:
# 結合したいCSVファイルが存在するディレクトリを指定
directory = './automation_med_best_result/'

# 結合したファイルを保存するためのファイル名
output_file = 'automation_median_combined_reviews.csv'

# "re"で始まり"review.csv"で終わるファイルをリストに格納
csv_files = [f for f in os.listdir(directory) if f.startswith('re') and f.endswith('review.csv')]

# 空のデータフレームを作成
combined_df = pd.DataFrame()

# ファイルを順に読み込み、データフレームに結合
for file in csv_files:
    file_path = os.path.join(directory, file)
    df = pd.read_csv(file_path)
    combined_df = pd.concat([combined_df, df], ignore_index=True)

# 'トータル所要時間'が675を超えるデータを削除
combined_df = combined_df[combined_df['トータル所要時間'] <= 675]

# 結合結果をCSVファイルとして保存
combined_df.to_csv(os.path.join(directory, output_file), index=False)

print(f"結合されたCSVファイルは {output_file} に保存されました。")


結合されたCSVファイルは automation_median_combined_reviews.csv に保存されました。


In [5]:
import os
import pandas as pd

# 複数のフォルダパスをリストで指定
directories = [
    './automation_best_result_csv/',
    './automation_med_best_result/',
    './best_result_csv/',
    # 必要に応じてフォルダパスを追加
]

# 出力先のファイル名を指定
output_file = 'combined_data.csv'

# 空のデータフレームを作成
combined_df = pd.DataFrame()

# 指定されたすべてのフォルダを走査
for directory in directories:
    # サブディレクトリも含めてCSVファイルを検索
    for root, _, files in os.walk(directory):
        for file in files:
            if file.endswith('.csv'):
                file_path = os.path.join(root, file)
                df = pd.read_csv(file_path)
                
                # 'トータル所要時間'が675以下のデータのみを抽出
                df_filtered = df[df['トータル所要時間'] <= 675]
                
                # フィルタリングされたデータを結合
                combined_df = pd.concat([combined_df, df_filtered], ignore_index=True)

# 結合したデータを1つのCSVファイルとして保存
combined_df.to_csv(output_file, index=False)

print(f"フィルタリングされたCSVファイルは {output_file} に保存されました。")

フィルタリングされたCSVファイルは combined_data.csv に保存されました。
